<a href="https://colab.research.google.com/github/andreydesousa92-byte/ML_models/blob/main/Intermitent_Sales_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Describbing Business Case
Sales & order data for Northwind Traders, a fictitious gourmet food supplier, including information on customers, products, orders, shippers, and employees.


## Our Goal!

To predic at material level if next month there will be any sales or not

# Importing databases

In [ ]:
import pandas as pd

In [10]:
# CATEGORIES
url = "https://raw.githubusercontent.com/andreydesousa92-byte/ML_models/main/categories.csv"
df_categories = pd.read_csv(url)
df_categories.head()

,categoryID,categoryName,description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains & Cereals,"Breads, crackers, pasta, and cereal"


In [12]:
# CUSTOMERS
url = "https://raw.githubusercontent.com/andreydesousa92-byte/ML_models/main/customers.csv"
df_customers = pd.read_csv(url, encoding='latin1')
df_customers.head()

,customerID,companyName,contactName,contactTitle,city,country
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Berlin,Germany
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Mexico City,Mexico
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mexico City,Mexico
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,London,UK
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Luleå,Sweden


In [32]:
# ORDERDETAILS
url = "https://raw.githubusercontent.com/andreydesousa92-byte/ML_models/main/order_details.csv"
df_salesorders = pd.read_csv(url, encoding='latin1')
df_salesorders.head()

,orderID,productID,unitPrice,quantity,discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [18]:
# ORDERDETAILS
url = "https://raw.githubusercontent.com/andreydesousa92-byte/ML_models/main/orders.csv"
df_orders = pd.read_csv(url)
df_orders.head()


,orderID,customerID,employeeID,orderDate,requiredDate,shippedDate,shipperID,freight
0,10248,VINET,5,2013-07-04,2013-08-01,2013-07-16,3,32.38
1,10249,TOMSP,6,2013-07-05,2013-08-16,2013-07-10,1,11.61
2,10250,HANAR,4,2013-07-08,2013-08-05,2013-07-12,2,65.83
3,10251,VICTE,3,2013-07-08,2013-08-05,2013-07-15,1,41.34
4,10252,SUPRD,4,2013-07-09,2013-08-06,2013-07-11,2,51.30


In [24]:
# Min and Max dates
min_date = df_orders['orderDate'].min()
max_date = df_orders['orderDate'].max()

print("Min order date:", min_date)
print("Max order date:", max_date)

Min order date: 2013-07-04
Max order date: 2015-05-06


In [25]:
# Create date range
calendar_df = pd.DataFrame({'Date': pd.date_range(start=min_date, end=max_date)})

# Extract year and month as separate columns
calendar_df['Year'] = calendar_df['Date'].dt.year
calendar_df['Month'] = calendar_df['Date'].dt.month
calendar_df['Day'] = calendar_df['Date'].dt.day

calendar_df.head(10)

,Date,Year,Month,Day
0,2013-07-04,2013,7,4
1,2013-07-05,2013,7,5
2,2013-07-06,2013,7,6
3,2013-07-07,2013,7,7
4,2013-07-08,2013,7,8
5,2013-07-09,2013,7,9
6,2013-07-10,2013,7,10
7,2013-07-11,2013,7,11
8,2013-07-12,2013,7,12
9,2013-07-13,2013,7,13


In [28]:
# ORDERDETAILS
url = "https://raw.githubusercontent.com/andreydesousa92-byte/ML_models/main/products.csv"
df_products = pd.read_csv(url, encoding='latin1')
df_products.head()

,productID,productName,quantityPerUnit,unitPrice,discontinued,categoryID
0,1,Chai,10 boxes x 20 bags,18.00,0,1
1,2,Chang,24 - 12 oz bottles,19.00,0,1
2,3,Aniseed Syrup,12 - 550 ml bottles,10.00,0,2
3,4,Chef Anton's Cajun Seasoning,48 - 6 oz jars,22.00,0,2
4,5,Chef Anton's Gumbo Mix,36 boxes,21.35,1,2


# Transform
The first step is to connect the databases, using the sales orders database as the core source, and transform the data into features that will be useful for our model.
We are using sales orders instead of billed sales because orders better reflect the actual demand requested by customers.

In [43]:
!pip install duckdb
import duckdb

result_df  = duckdb.query("""
WITH
calendar AS(
    SELECT
        MIN(Date) AS Date,
        Year AS sales_year,
        Month AS sales_month
    FROM calendar_df
    group by
        Year,
        Month
),
parts_orgs AS (
    SELECT DISTINCT
        productID AS keyId
    FROM df_products
),
sales AS (
    SELECT
        YEAR(CAST(o.orderDate AS DATE)) AS sales_year,
        MONTH(CAST(o.orderDate AS DATE)) AS sales_month,
        so.productID AS keyId,
        SUM(so.quantity) AS Sales_EA,
        COUNT(*) AS Num_Sales_Orders,
        COUNT(DISTINCT o.customerID) AS Num_Of_Customers
    FROM df_orders o
    LEFT JOIN df_salesorders so
      ON o.OrderID = so.OrderID
    GROUP BY
        YEAR(CAST(o.orderDate AS DATE)) ,
        MONTH(CAST(o.orderDate AS DATE)),
        so.productID
    HAVING SUM(so.quantity) > 0
)
SELECT
    g.Date,
    g.sales_year,
    g.sales_month,
    g.keyId,
    COALESCE(s.Sales_EA, 0) AS Sales_EA,

    -- CAPTURES NEXT MONTH SALES
    LEAD(COALESCE(s.Sales_EA, 0)) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
    ) AS Next_Month_Sales_EA,

    -- BINARY TO INDICATE IF NEXT MONTH THERE WILL BE SALES OR NOT
    CASE
        WHEN
        LEAD(COALESCE(s.Sales_EA, 0)) OVER (
            PARTITION BY g.keyId
            ORDER BY g.sales_year, g.sales_month
        ) > 0
        THEN 1
        ELSE 0
    END AS Have_Sales_NMonth,

    -- SIN AND COS ARE RELEVANT FOR SEASONALITY
    SIN(2 * PI() * g.sales_month / 12) AS month_sin,
    COS(2 * PI() * g.sales_month / 12) AS month_cos,

    -- SALES LAGS
    LAG(COALESCE(s.Sales_EA, 0), 1) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
    ) AS Sales_Lag_1,
    LAG(COALESCE(s.Sales_EA, 0), 2) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
    ) AS Sales_Lag_2,
    LAG(COALESCE(s.Sales_EA, 0), 3) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
    ) AS Sales_Lag_3,

    -- ROLLING SUMS
    SUM(COALESCE(s.Sales_EA, 0)) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Sum_Last_3_Months,

    -- MONTHS SINCE LAST SALES
    CASE
        WHEN MAX(CASE WHEN COALESCE(s.Sales_EA, 0) > 0 THEN (g.sales_year * 12 + g.sales_month) END)
             OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month
                   ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) IS NULL
        THEN 999
        ELSE (g.sales_year * 12 + g.sales_month) -
             MAX(CASE WHEN COALESCE(s.Sales_EA, 0) > 0 THEN (g.sales_year * 12 + g.sales_month) END)
                 OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month
                       ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    END AS months_since_last_sale,

    -- L2M WITHOUT SALES RULE
    CASE
        WHEN
            LAG(COALESCE(s.Sales_EA, 0), 1) OVER (
                PARTITION BY g.keyId
                ORDER BY g.sales_year, g.sales_month
            ) = 0 AND
            LAG(COALESCE(s.Sales_EA, 0), 2) OVER (
                PARTITION BY g.keyId
                ORDER BY g.sales_year, g.sales_month
            ) =0
        THEN 1
        ELSE 0
    END AS no_sales_L2M,

    -- SALES TREND
    COALESCE(LAG(s.Sales_EA,1) OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month),0) -
    COALESCE(LAG(s.Sales_EA,2) OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month),0)
    AS Lag_Trend_1_2,

    COALESCE(LAG(s.Sales_EA,2) OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month),0) -
    COALESCE(LAG(s.Sales_EA,3) OVER (PARTITION BY g.keyId ORDER BY g.sales_year, g.sales_month),0)
    AS Lag_Trend_2_3,

    -- PERCENTAGE OF MONTHS WITH SALES
    SUM(
        CASE WHEN
            COALESCE(s.Sales_EA,0) > 0
        THEN 1 ELSE 0 END
    ) OVER (PARTITION BY g.keyId)
    /
    COUNT(*) OVER (PARTITION BY g.keyId)
    AS Product_Activity_Ratio,

    -- STANDARD DEVIATION OF LAST 6 MONTHS
    STDDEV(COALESCE(s.Sales_EA,0)) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) AS Sales_Std_6M,

    -- 6M ACITIVY RATE
    SUM(
        CASE WHEN COALESCE(s.Sales_EA,0) > 0 THEN 1 ELSE 0 END
    ) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) / 6.0 AS Activity_Rate_6M,

    -- 3M ACITIVY RATE
    SUM(
        CASE WHEN COALESCE(s.Sales_EA,0) > 0 THEN 1 ELSE 0 END
    ) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) / 3.0 AS Activity_Rate_3M,

    -- RECENCY INTERACTION
    Product_Activity_Ratio * months_since_last_sale AS Activity_Recency_Interaction,

    -- SEASONALITY INTERACTION
    month_sin * Product_Activity_Ratio AS Season_Activity_Interaction,

    -- NUMBER OF CUSTOMERS ON LAST 6M
    SUM(COALESCE(s.Num_Of_Customers,0)) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) AS Num_Customers_L6M,

    -- NUMBER OF SALES ORDERS
    SUM(COALESCE(s.Num_Sales_Orders,0)) OVER (
        PARTITION BY g.keyId
        ORDER BY g.sales_year, g.sales_month
        ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) AS Num_SalesOrders_L6M

FROM (
    SELECT
        c.Date,
        c.sales_year,
        c.sales_month,
        p.keyId
    FROM calendar c
    CROSS JOIN parts_orgs p
) g
LEFT JOIN sales s
    ON g.sales_year = s.sales_year
   AND g.sales_month = s.sales_month
   AND g.keyId = s.keyId
ORDER BY
    g.keyId,
    g.sales_year,
    g.sales_month;
""").to_df()

In [44]:
result_df.head()

,Date,sales_year,sales_month,keyId,Sales_EA,Next_Month_Sales_EA,Have_Sales_NMonth,month_sin,month_cos,Sales_Lag_1,...,Lag_Trend_1_2,Lag_Trend_2_3,Product_Activity_Ratio,Sales_Std_6M,Activity_Rate_6M,Activity_Rate_3M,Activity_Recency_Interaction,Season_Activity_Interaction,Num_Customers_L6M,Num_SalesOrders_L6M
0,2013-07-04,2013,7,1,0.0,63.0,1,-0.500000,-8.660254e-01,NaN,...,0.0,0.0,0.782609,NaN,0.000000,0.000000,781.826087,-0.391304,0.0,0.0
1,2013-08-01,2013,8,1,63.0,20.0,1,-0.866025,-5.000000e-01,0.0,...,0.0,0.0,0.782609,44.547727,0.166667,0.333333,0.000000,-0.677759,2.0,2.0
2,2013-09-01,2013,9,1,20.0,0.0,0,-1.000000,-1.836970e-16,63.0,...,63.0,0.0,0.782609,32.192132,0.333333,0.666667,0.000000,-0.782609,3.0,3.0
3,2013-10-01,2013,10,1,0.0,27.0,1,-0.866025,5.000000e-01,20.0,...,-43.0,63.0,0.782609,29.702693,0.333333,0.666667,0.782609,-0.677759,3.0,3.0
4,2013-11-01,2013,11,1,27.0,15.0,1,-0.500000,8.660254e-01,0.0,...,-20.0,-43.0,0.782609,25.874698,0.500000,0.666667,0.000000,-0.391304,5.0,5.0
